In [1]:
from typing import Annotated

from rdflib import Graph, URIRef
from autogen import ConversableAgent

import sys
sys.path.append('../../..')
from inswitch.llm.model import get_openai_model_config

In [29]:
def query_service_desc(
) -> Annotated[str, "Service descriptions in RDF turtle format"]:
    
    kg = Graph()
    kg.parse("../data/anonymized_data_fill.ttl")
    select_query = """
    PREFIX fill: <https://intendproject.eu/fill/>
    PREFIX intend: <https://intendproject.eu/schema/>
    PREFIX schema: <https://schema.org/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?service ?p ?description WHERE {    
	    ?service ?p ?description FILTER(?p = intend:description)
    }
    """
    qres = kg.query(select_query)
    triples = []
    for row in qres:
        triple = []
        for item in row:
            item_prefix = kg.namespace_manager.qname(item) if isinstance(item, URIRef) else f'"{item}"'
            triple.append(item_prefix)
        triple.append('.')
        triple = ' '.join(triple)
        triples.append(triple)
    return '\n'.join(triples)

In [20]:
def find_containers(
        active_machine: Annotated[str, "Active machine"],
        service: Annotated[str, "Service"]
) -> Annotated[str, "Name of containers that deploy the service on the active machine"]:
    
    service = "fill:" + service
    active_machine = "fill:" + active_machine
    kg = Graph()
    kg.parse("../data/anonymized_data_fill.ttl")
    select_query = f"""
    PREFIX fill: <https://intendproject.eu/fill/>
    PREFIX intend: <https://intendproject.eu/schema/>
    PREFIX schema: <https://schema.org/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

    SELECT ?container WHERE {{    
	    {active_machine} intend:is_machine_type ?machine_type .
        ?machine_type intend:hasService ?service FILTER(?service = {service})
        ?service intend:hasComponent ?component .
        ?component intend:hasContainer ?container .
        ?container intend:canBeDeployedOn ?machine_type .
        
    }}
    """

    qres = kg.query(select_query)
    containers = []
    if qres:
            for row in qres:
                containers.append(row[0].replace('https://intendproject.eu/fill/',''))
            return ', '.join(containers)
    else:
        return "Can't find containers"

# print(find_containers('Active_Machine6', 'Service5'))

In [30]:
service_finder = ConversableAgent(
    name="Service_Finder",
    system_message="Find the relevant service based on the customer's question.",
    llm_config={"config_list": [get_openai_model_config()]},
    human_input_mode="NEVER"
)

container_finder = ConversableAgent(
    name="Container_finder",
    system_message="Find the relevant containers based on the customer's active machine and service.",
    llm_config={"config_list": [get_openai_model_config()]},
    human_input_mode="NEVER"
)

user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    human_input_mode="NEVER",
)

In [31]:
service_finder.register_for_llm(name="query_service_desc", 
                               description="Returns service descriptions as RDF triples in turtle format")(query_service_desc)

user_proxy.register_for_execution(name="query_service_desc")(query_service_desc)

<function __main__.query_service_desc() -> Annotated[str, 'Service descriptions in RDF turtle format']>

In [32]:
container_finder.register_for_llm(name="find_containers",
                               description="Returns the containers based on the customer's active machine and service")(find_containers)

user_proxy.register_for_execution(name="find_containers")(find_containers)

<function __main__.find_containers(active_machine: Annotated[str, 'Active machine'], service: Annotated[str, 'Service']) -> Annotated[str, 'Name of containers that deploy the service on the active machine']>

In [33]:
user_intent = '''I am Customer1 and own Active_Machine6. 
I want to add a new service to my machine to record all the alarms.
'''

chat_results = user_proxy.initiate_chats(
    [
         {
            "recipient": service_finder,
            "message": user_intent,
            "max_turns": 2,
            "summary_method": "last_msg",
        },

          {
            "recipient": container_finder,
            "message": user_intent,
            "max_turns": 2,
            "summary_method": "last_msg",
        }
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
User (to Service_Finder):

I am Customer1 and own Active_Machine6. 
I want to add a new service to my machine to record all the alarms.


--------------------------------------------------------------------------------
Service_Finder (to User):

***** Suggested tool call (call_KrVHRGlnF21f6MKLYl7IXd2c): query_service_desc *****
Arguments: 
{}
***********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION query_service_desc...
User (to Service_Finder):

User (to Service_Finder):

***** Response from calling tool (call_KrVHRGlnF21f6MKLYl7IXd2c) *****
fill:Service5 intend:description "Service5 records all alarms and messages and identifies the main messages. Actual machine p